In [2]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt
import GPy
import os
import matplotlib as mpl
import matplotlib.tri as tri
import ternary
import pickle
import datetime
from collections import Counter
import matplotlib.ticker as ticker
from sklearn import preprocessing
import pyDOE
import random
from scipy.stats import norm
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import copy

# Load materials dataset

In [3]:
class Data_Handler():
    def __init__(self, dataset_name, maximization=True):

        # Parameters:
        # dataset_name (string)
        #       name of the dataset to use. Options include ['Crossed barrel', 'Perovskite', 'AgNP', 'P3HT', 'AutoAM']
        # maximization (boolean)
        #       If true then the objective value will be positive (maximization objective problem, use for P3HT/CNT, Crossed barrel, AutoAM)
        #       If false then the objective value will be negative (minimzation objective problem, use for Perovskite, AgNP)

        self.dataset = self.load_data(dataset_name)
        self.normalized_dataset = self.process_data(maximization)


    def load_data(self, dataset_name):

        # Parameters:
        # dataset_name (string)
        #       name of the dataset to use. Options include ['Crossed barrel', 'Perovskite', 'AgNP', 'P3HT', 'AutoAM']
        # Outputs:
        # dataset (pd.DataFrame)
        #       dataset in the form of a pandas dataframe

        raw_dataset = pd.read_csv('datasets/' + dataset_name + '_dataset.csv')
        dataset = copy.deepcopy(raw_dataset) 

        return dataset
    

    def process_data(self, maximization):

        # Parameters:
        # maximization (boolean)
        #       If true then the objective value will be positive (maximization objective problem, use for P3HT/CNT, Crossed barrel, AutoAM)
        #       If false then the objective value will be negative (minimzation objective problem, use for Perovskite, AgNP)
        # Outputs:
        # grouped_dataset (pd.DataFrame)
        #       to perform pool-based active learning, need to group the data by unique input feature x value
        #       for each unique x in design space, only keep the average of all evaluations there as its objective value

        feature_names, objective_name = list(self.dataset.columns)[:-1], list(self.dataset.columns)[-1]

        if maximization is True:
            self.dataset[objective_name] = self.dataset[objective_name].values
        if maximization is False:
            self.dataset[objective_name] = -self.dataset[objective_name].values
    
        processed_dataset = self.dataset.groupby(feature_names)[objective_name].agg(lambda x: x.unique().mean())
        processed_dataset = (processed_dataset.to_frame()).reset_index()

        return processed_dataset
    
    
    def get_top_n_indices(self, top_percentage):

        # Parameters:
        # top_percentage (float)
        #       Denotes the top_percentage of data samples as being one of the top samples in the dataset
        #       Ex. top_percentage = 0.05 will retrieve the indices of the top 5% of data samples
        # Outputs:
        # top_indices (list)
        #       list corresponding to the indices of belonging to data samples in the top_percentage of all samples in the dataset

        objective_name = list(self.dataset.columns)[-1]

        n_top = int(math.ceil(len(self.normalized_dataset) * top_percentage))
        top_indices = list(self.normalized_dataset.sort_values(objective_name).head(n_top).index)

        return top_indices
    

    def X_y_split(self):

        # Outputs:
        # X, y (np.array), (np.array)
        #       X, numpy array containing data from the input features
        #       y, numpy array containing data from the output feature 

        feature_names, objective_name = list(self.dataset.columns)[:-1], list(self.dataset.columns)[-1]

        X = self.normalized_dataset[feature_names].values
        y = np.array(self.normalized_dataset[objective_name].values)

        assert len(self.normalized_dataset) == len(X) == len(y), "X and y have a different amount of rows"
        return X, y



# Create Surrogate Models

In [11]:
class RF_Surrogate_Model():
    def __init__(self, n_est = 100, n_jobs = -1):

        # Parameters:
        # n_est (integer)
        #       the number of decision trees to be used in Random Forests Model
        #       Paper specifies 100 is suitable for all 5 datasets
        # n_jobs (integer)
        #       The number of jobs to run in parallel
        #       -1 denotes using all processors

        self.model = RandomForestRegressor(n_estimators = n_est, n_jobs = n_jobs)
        self.n_est = n_est

    def fit(self, X, y):

        # Parameters:
        # X (np.array)
        #       training data from X to train the model on
        # y (np.array)
        #       training data from y to train the model on

        self.model.fit(X, y)

    def predict(self, X):

        # Parameters:
        # X (np.array)
        #       Data from X for the model to form predictions on
        # Outputs:
        # mean, std (float), (float)
        #       mean of the models predictions
        #       std of the models predictions

        tree_predictions = []

        for j in np.arange(self.n_est):
            tree_predictions.append((self.model.estimators_[j].predict(np.array([X]))).tolist())

        mean = np.mean(np.array(tree_predictions), axis=0)[0]
        std = np.std(np.array(tree_predictions), axis=0)[0]

        return mean, std
    
    
    

    
class GP_Surrogate_Model():
    def __init__(self, X, kernel_name, ARD_):

        # Parameters:
        # X (np.array)
        #       X dataset, used to initialize different kernels
        # kernel_name (str)
        #       string denoting which kernel to use. Options include ['Mat52', 'Mat32', 'Mat12', 'RBF']
        # ARD_ (boolean)
        #       If true, initializes GP model with anisotropic kernels
        #       If flase, initializes GP model with isotropic kernels

        kernel_ = self.get_kernel(X, kernel_name, ARD_)

        self.model = GPyOpt.models.GPModel(
            optimize_restarts = 10,
            kernel = kernel_,
            noise_var = 0.01,
            max_iters = 100,
            ARD = ARD_,
            optimizer = 'bfgs',
            verbose = False)


    def get_kernel(self, X, kernel_name, ARD_):

        # Parameters:
        # X (np.array)
        #       X dataset, used to initialize different kernels
        # kernel_name (str)
        #       string denoting which kernel to use. Options include ['Mat52', 'Mat32', 'Mat12', 'RBF']
        # ARD_ (boolean)
        #       If true, initializes GP model with anisotropic kernels
        #       If flase, initializes GP model with isotropic kernels
        # Outputs:
        # kernel (GPy.kern object)
        #       returns kernel corresponding to kernel_name

        Bias_k = GPy.kern.Bias(X.shape[1], variance=1.)

        Matern52_k = GPy.kern.Matern52(X.shape[1], variance=1., ARD=ARD_) + Bias_k
        Matern32_k = GPy.kern.Matern32(X.shape[1], variance=1., ARD=ARD_) + Bias_k
        Matern12_k = GPy.kern.Exponential(X.shape[1], variance=1., ARD=ARD_) + Bias_k
        RBF_k = GPy.kern.RBF(X.shape[1], variance=1., ARD=ARD_) + Bias_k

        kernels = {"Mat52" : Matern52_k, 
                   "Mat32" : Matern32_k,
                   "Mat12" : Matern12_k, 
                   "RBF" : RBF_k}

        return kernels[kernel_name]


    def fit(self, X, y):

        # Parameters:
        # X (np.array)
        #       training data from X to train the model on
        # y (np.array)
        #       training data from y to train the model on

        self.model.updateModel(X, y, None, None)


    def predict(self, X):
        
        # Parameters:
        # X (np.array)
        #       Data from X for the model to form predictions on
        # Outputs:
        # mean, std (float), (float)
        #       mean of the models predictions
        #       std of the models predictions

        mean, var = self.model.predict(X)

        return mean, np.sqrt(var)

# Run Bayesian Optimization

In [12]:
class BO():
    def __init__(self, X, y, top_indices):

        # Parameters:
        # X (np.array)
        #       X dataset of input features
        # Y (np.array)
        #       y dataset of the output label
        # top_indices (list)
        #       Corresponds to the indices of the top candidates defined earlier
        #       Allows for keeping track of if a top candidate has been found

        self.X = X
        self.y = y
        self.top_indices = top_indices

        self.index_collection = []
        self.X_collection = []
        self.y_collection = []
        self.TopCount_collection = []


    # expected improvement
    def EI(self, X, model, y_best):

        # Parameters:
        # X (np.array)
        #       Data samples from X
        # model (RF_Surrogate_Model or GP_Surrogate_Model Object)
        #       model object, either GP or RF
        # y_best (float)
        #       Denotes the maximum value in y of the explored data samples in y
        # Outputs:
        # (integer)
        #       Denotes the next X value to explore/exploit     
        
        xi = 0 # can also use 0.01
        mean, std = model.predict(X)

        z = (y_best - mean - xi)/std
        return (y_best - mean - xi) * norm.cdf(z) + std * norm.pdf(z)


    # lower confidence bound
    def LCB(self, X, model, ratio):

        # Parameters:
        # X (np.array)
        #       Data samples from X
        # model (RF_Surrogate_Model or GP_Surrogate_Model Object)
        #       model object, either GP or RF
        # ratio (integer)
        #       parameter controls whether the acquisition function should focus one exploration or exploitation
        #       higher ratio means more focus on exploration
        #       lower ratio means more focus on exploitation
        # Outputs:
        # (integer)
        #       Denotes the next X value to explore/exploit  

        mean, std = model.predict(X)
        
        return - mean + ratio * std


    # probability of improvement
    def PI(self, X, model, y_best):

        # Parameters:
        # X (np.array)
        #       Data samples from X
        # model (RF_Surrogate_Model or GP_Surrogate_Model Object)
        #       model object, either GP or RF
        # y_best (float)
        #       Denotes the maximum value in y of the explored data samples in y
        # Outputs:
        # (integer)
        #       Denotes the next X value to explore/exploit  

        xi = 0 # can also use 0.01
        mean, std = model.predict(X)
        
        z = (y_best - mean - xi)/std
        return norm.cdf(z)
    
    def find_next_index(self, indices_to_explore, model, acqusition_function_name, y_best):
        next_index = None

        #initialize negative number to ensure max_ac gets overwritten in following for loop
        max_ac = -10**10

        for j in indices_to_explore:
            X_j = self.X[j]
            y_j = self.y[j]

            # select Acquisiton Function for BO
            if acqusition_function_name == "LCB":
                ac_value = self.LCB(X_j, model, 2)

            elif acqusition_function_name == "EI":
                ac_value = self.EI(X_j, model, y_best)

            elif acqusition_function_name == "PI":
                ac_value = self.PI(X_j, model, y_best)

            # in BO explore index with largest acquisition function value
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j

        # return best index to explore/exploit according to acquisition function
        return next_index
    
    
    def run_bayesian_optimization(self, 
                                  n_ensemble = 50, 
                                  n_initial = 2,
                                  surrogate_model_name = "RF",
                                  acqusition_function_name = "LCB",
                                  **kwargs):
        
        # Parameters:
        # n_ensemble (integer)
        #       The number of ensemble models to run, paper uses 50
        # n_initial (integer)
        #       The number of initial experiments to run, paper uses 2
        # surrogate_model_name (string)
        #       The name of the surrogate model to use. Options include  ['RF', 'GP']
        # acquisition_function_name
        #       The name of the acquisition function to use. Options include  ['LCB', 'EI', 'PI']
        # Outputs:
        # (npy file)
        #       Returns npy file of results from bayesian optimization algorithm
        
        start_time = time.time()
        seed_list = seed_list = [4295, 8508, 326, 3135, 1549, 2528, 1274, 6545, 5971, 6269, 2422, 4287, 9320, 4932, 951, 4304, 1745, 5956, 7620, 4545, 6003, 9885, 5548, 9477, 30, 8992, 7559, 5034, 9071, 6437, 3389, 9816, 8617, 3712, 3626, 1660, 3309, 2427, 9872, 938, 5156, 7409, 7672, 3411, 3559, 9966, 7331, 8273, 8484, 5127, 2260, 6054, 5205, 311, 6056, 9456, 928, 6424, 7438, 8701, 8634, 4002, 6634, 8102, 8503, 1540, 9254, 7972, 7737, 3410, 4052, 8640, 9659, 8093, 7076, 7268, 2046, 7492, 3103, 3034, 7874, 5438, 4297, 291, 5436, 9021, 3711, 7837, 9188, 2036, 8013, 6188, 3734, 187, 1438, 1061, 674, 777, 7231, 7096, 3360, 4278, 5817, 5514, 3442, 6805, 6750, 8548, 9751, 3526, 9969, 8979, 1526, 1551, 2058, 6325, 1237, 5917, 5821, 9946, 5049, 654, 7750, 5149, 3545, 9165, 2837, 5621, 6501, 595, 3181, 1747, 4405, 4480, 4282, 9262, 6219, 3960, 4999, 1495, 6007, 9642, 3902, 3133, 1085, 3278, 1104, 5939, 7153, 971, 8733, 3785, 9056, 2020, 7249, 5021, 3384, 8740, 4593, 7869, 9941, 8813, 3688, 8139, 6436, 3742, 5503, 1587, 4766, 9846, 9117, 7001, 4853, 9346, 4927, 8480, 5298, 4753, 1151, 9768, 5405, 6196, 5721, 3419, 8090, 8166, 7834, 1480, 1150, 9002, 1134, 2237, 3995, 2029, 5336, 7050, 6857, 8794, 1754, 1184, 3558, 658, 6804, 8750, 5088, 1136, 626, 8462, 5203, 3196, 979, 7419, 1162, 5451, 6492, 1562, 8145, 8937, 8764, 4174, 7639, 8902, 7003, 765, 1554, 6135, 1689, 9530, 1398, 2273, 7925, 5948, 1036, 868, 4617, 1203, 7680, 7, 93, 3128, 5694, 6979, 7136, 8084, 5770, 9301, 1599, 737, 7018, 3774, 9843, 2296, 2287, 9875, 2349, 2469, 8941, 4973, 3798, 54, 2938, 4665, 3942, 3951, 9400, 3094, 2248, 3376, 1926, 5180, 1773, 3681, 1808, 350, 6669, 826, 539, 5313, 6193, 5752, 9370, 2782, 8399, 4881, 3166, 4906, 5829, 4827, 29, 6899, 9012, 6986, 4175, 1035, 8320, 7802, 3777, 6340, 7798, 7705]

        for i in range(n_ensemble):
            seed = seed_list[i]

            print('Initializing seed = ' + str(seed_list.index(seed)))
            random.seed(seed)

            # the pool of candidates to be examined
            indices_to_explore = list(np.arange(len(self.X))).copy()
            # the list of candidates we have already observed (add initial experiments)
            observed_indices = random.sample(indices_to_explore, n_initial)

            X_, y_, TopCount_ = [], [], []
            top_indices_count = 0

            # check if initial experiments contained top candidates
            for i in observed_indices:
                X_.append(self.X[i])
                y_.append(self.y[i])

                # if so increment top_indices count
                if i in self.top_indices:
                    top_indices_count += 1

                # remove said indices from the list of indices to be explored
                TopCount_.append(top_indices_count)
                indices_to_explore.remove(i)

            # this for loop ends when all candidates in pool are observed
            for i in np.arange(len(indices_to_explore)):
                # as AutoAM is a maximization optimization problem y_best will be the maximum y value examined so far
                y_best = np.max(y_)

                s_scaler = StandardScaler()
                X_train = s_scaler.fit_transform(X_)
                y_train = s_scaler.fit_transform([[i] for i in y_])

                # select Surrogate Model for BO
                if surrogate_model_name == "RF":
                    n_est = kwargs.get("n_est", 100)
                    model = RF_Surrogate_Model(n_est, n_jobs=-1)
                    model.fit(X_train, y_train)

                elif surrogate_model_name == "GP":
                    kernel_name = kwargs.get("kernel_name", "Mat52")
                    ARD_ = kwargs.get("ARD", True)

                    try:
                        model = GP_Surrogate_Model(self.X, kernel_name, ARD_)
                        model.fit(X_train, y_train)

                    except:
                        break

                # calculate the next index to explore according to the acquisition function
                next_index = self.find_next_index(indices_to_explore, model, acqusition_function_name, y_best)

                X_.append(self.X[next_index])
                y_.append(self.y[next_index])

                # if next_index corresponds to a top candidate increment top candidates count
                if next_index in self.top_indices:
                    top_indices_count += 1

                # Append top_indices_count, shows the number of top candidates found each ensemble
                TopCount_.append(top_indices_count)

                indices_to_explore.remove(next_index)
                observed_indices.append(next_index)

            try:
                assert len(observed_indices) == len(self.X)

                self.index_collection.append(observed_indices)
                self.X_collection.append(X_)
                self.y_collection.append(y_)
                self.TopCount_collection.append(TopCount_)

            except:
                # Occurs in some cases of BO with GP, randomly code fails the above assertion
                print('Error during train, moving on to next seed')

            print('Finished seed')

        total_time = time.time() - start_time

        master = np.array([self.index_collection, self.X_collection, self.y_collection, self.TopCount_collection, total_time])

        # TODO: Name output file
        np.save(surrogate_model_name + "_final_run", master)


# Run Bayesian Optimization Code

In [14]:
# load a dataset
# dataset names = ['Crossed barrel', 'Perovskite', 'AgNP', 'P3HT', 'AutoAM']
data_handler_object = Data_Handler('AutoAM')

top_indices = data_handler_object.get_top_n_indices(0.05)
X, y = data_handler_object.X_y_split()

bayesian_optimization = BO(X, y, top_indices)

# to run the code with Random Forests (roughly 1 hour runtime at 50 ensembles)
bayesian_optimization.run_bayesian_optimization(n_ensemble = 10, 
                                                n_initial = 2, 
                                                surrogate_model_name = "RF",
                                                acqusition_function_name = "LCB", 
                                                n_est = 100)

# to run the code with Gaussian Process with ARD (roughly 2.5 hours runtime at 50 ensembles)
# bayesian_optimization.run_bayesian_optimization(n_ensemble = 10, 
#                                                 n_initial = 2, 
#                                                 surrogate_model_name = "GP", 
#                                                 acqusition_function_name = "LCB", 
#                                                 kernel_name = "Mat52",
#                                                 ARD = True)

Initializing seed = 0
Finished seed
Initializing seed = 1
Finished seed
Initializing seed = 2
Finished seed
Initializing seed = 3
Finished seed
Initializing seed = 4
Finished seed
Initializing seed = 5
Finished seed
Initializing seed = 6
Finished seed
Initializing seed = 7
Finished seed
Initializing seed = 8
Finished seed
Initializing seed = 9
Finished seed
